In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #Data Visualization 
import seaborn as sns  #Python library for Vidualization
from sklearn.cluster import KMeans

In [2]:
# defining calculation functions

def calcBA(pruns, pinnings, pnotout):
    ba = pruns / (pinnings - pnotout)
    return ba


def calcBS(pruns, pbfaced):
    bs = pruns / pbfaced
    return bs


def calcMRA(pinnings, pnotout, pbfaced):
    mra = (pinnings - pnotout) / pbfaced
    return mra


def calcBRPI(pfours, psixes, pinnings):
    brpi = (4*pfours + 6*psixes) / pinnings
    return brpi


def calcBatOutRate(pinnings, pnotout, pbfaced):
    outrate = pd.Series(np.array([0])) if float(pbfaced) == 0 else (pinnings - pnotout) / pbfaced
    return outrate

#         BA = calcBA(p['runs'], p['innings'], p['not_out'])
#         BS = calcBS(p['runs'], p['ball_faced'])
#         MRA = calcMRA(p['innings'], p['not_out'], p['ball_faced'])
#         BRPI = calcBRPI(p['fours'], p['sixes'], p['innings'])


def calcGenBatAvg(truns, twickets):
    return truns / twickets


def calcGenBatOutRate(twickets, tbfaced):
    return twickets / tbfaced


def calcGenStrikeRate(truns, tbfaced):
    return truns / tbfaced


def calcBatAGR(tpRuns, tpBallfaced, tpOutRate, gBatAvg, gStrikeRate, gOutRate):
    agr = ((tpRuns - gStrikeRate * tpBallfaced) + gBatAvg * tpBallfaced * (gOutRate - tpOutRate))
#     d = np.array([132])
#     tm = pd.Series(np.array([0]))
    return agr


def calcBatRankingIndex(agr, gBatAvg):
    return agr / (10 * gBatAvg)


In [3]:
# defining bpwlling related calculation functions

# for each player

def calcBowlAvg(runsGiven, wicketsTaken):
    return runsGiven / wicketsTaken


def calcBowlSR(ballsBowled, wicketsTaken):
    return ballsBowled / wicketsTaken


def calcBowlER(runsGiven, oversBowled):
    return runsGiven / oversBowled


def calcBowlOutRate(wicketsTaken, ballsBowled):
    outr = pd.Series(np.array([0])) if float(ballsBowled) == 0 else wicketsTaken / ballsBowled
    return outr

# generic tournament parameters

def calcGenBowlAvg(totalRunsGiven, totalWicketsTaken):
    return totalRunsGiven / totalWicketsTaken


def calcGenBowlOutRate(totalWicketsTaken, totalOvers):
    balls = int(totalOvers) * 6 + 10 * (totalOvers - int(totalOvers))
#     d = np.array([132])
#     tm = pd.Series(0.12048192771084336)
    return totalWicketsTaken / balls
#     return tm


def calcGenBowlSR(totalRunsGiven, totalOvers):
    balls = int(totalOvers) * 6 + 10 * (totalOvers - int(totalOvers))
    return  totalRunsGiven / balls


def calcBowlAGR(tpRunsGiven, tpOvers, tpOutrate, gBowlAvg, gBowlSR, gBowlOutrate ):
    balls = int(tpOvers) * 6 + 10 * (tpOvers - int(tpOvers))
    bowlagr = (gBowlSR * balls - tpRunsGiven) + gBowlAvg * balls * (gBowlOutrate - tpOutrate)
    return bowlagr


def calcBowlRI(bowlAGR, gBowlAvg):
    return bowlAGR / (10 * gBowlAvg)

In [4]:
df = pd.read_csv("serieswiseMatchRecord.csv")
print(df.columns)

Index(['series_code', 'match_code', 'pcid', 'pcountry', 'pid', 'name',
       'notout', 'runs', 'ballfaced', 'fours', 'sixes', 'overs',
       'maiden_overs', 'runs_given', 'wickets_taken'],
      dtype='object')


In [7]:
series = list(dict.fromkeys(df["series_code"]).keys())
# series.sort(reverse=True)

# trecord : dataframe for tournament overall performance
# precord : dataframe for player specific performance in tournament
# tplayer: dataframe for recording total runs, ballfaced etc for player

# seriespdata = pd.DataFrame()
trecord = pd.DataFrame()
precord = pd.DataFrame()
tplayer = pd.DataFrame()
pindexes = pd.DataFrame()

tournamentPerformance = dict()

for sr in series[:]:
    tdf = df[df["series_code"] == sr]
 
    ttotal = tdf.drop(['series_code', 'match_code', 'pid', 'name', 'pcid',
                                               'pcountry'], axis = 1)
    
    ttotal = pd.DataFrame(ttotal.sum(axis = 0))
    ttotal = ttotal.swapaxes('index', 'columns')
    ttotal.insert(0, 'series_code', value = sr)
    
    gBatAvg = calcGenBatAvg(ttotal['runs'], ttotal['wickets_taken'])
    gBatOutRate = calcGenBatOutRate(ttotal['wickets_taken'], ttotal['ballfaced'])
    gStrikeRate = calcGenStrikeRate(ttotal['runs'], ttotal['ballfaced'])
    
    gBowlAvg = calcBowlAvg(ttotal['runs_given'], ttotal['wickets_taken'])
    gBowlOutrate = calcGenBowlOutRate(ttotal['wickets_taken'], ttotal['overs'])
    gBowlSR = calcGenBowlSR(ttotal['runs_given'], ttotal['overs'])
    
    tp = {
        'series_code' : sr, 
        'gBatAvg'     : gBatAvg[0], 
        'gBatOutRate' : gBatOutRate[0], 
        'gStrikeRate' : gStrikeRate[0],
        'gBowlAvg'    : gBowlAvg[0], 
        'gBowlOutrate' : gBowlOutrate[0], 
        'gBowlSR'     : gBowlSR[0]
    }
    
    trecord = trecord.append(tp, ignore_index=True)
    
    
#     print((tp))
# print(len(series))

    players = list()
    for pl in list(tdf["pid"]):
        if pl > 0:
            players.append(pl)
        
#     print('current ',players)
    
    for player in players[:]:
        
        
        ptdf = tdf[tdf['pid'] == player].drop(['series_code', 'match_code', 'pid', 'name', 'pcid',
                                               'pcountry'], axis = 1)
        
        pinnings = len(ptdf)
        p = pd.DataFrame(ptdf.sum(axis = 0))
        p = p.swapaxes('index', 'columns')
        
#         print(p)
        tplayer = tplayer.append(p)
    
        batOutRate = calcBatOutRate(pinnings, p['notout'], p['ballfaced'])
        bowlOutRate = calcBowlOutRate(p['wickets_taken'], p['overs'])
        
#         p.insert(0, 'innings' ,value = pinnings)
#         p.insert(0, 'pid' ,value = player)
#         p.insert(0, 'series_code' ,value = sr)
#         p.insert(4, 'outrate' ,value = outRate)

        batAGR = calcBatAGR(p['runs'], p['ballfaced'], batOutRate, gBatAvg, gStrikeRate, gBatOutRate)
        batRI = calcBatRankingIndex(batAGR, gBatAvg)
        
        bowlAGR = calcBowlAGR(p['runs_given'], p['overs'], bowlOutRate, gBowlAvg, gBowlSR, gBowlOutrate)
        bowlRI = calcBowlRI(bowlAGR, gBowlAvg)
        
        pRI = {
            'series_code' : sr,
            'pid'    : player,
            'batAGR' : batAGR[0],
            'batRI'  : batRI[0],
            'bowlAGR': bowlAGR[0],
            'bowlRI' : bowlRI[0]
        }
        
        pindexes = pindexes.append(pRI, ignore_index = True)
#         print(type(batAGR))
#         print(batOutRate[0], bowlOutRate[0])
#         print('batAGR ', batAGR[0], '                        batRI ', batRI[0])
#         print('bowlAGR ', bowlAGR[0], '                       bowlRI ', bowlRI[0])
#         print()
        

In [8]:
pindexes.to_csv('playerIR.csv', index = False)